In [1]:
%pip install tdt numpy matplotlib

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


Need to check the stream name, fiber photometry data is saved under specific stream names depending on your rig setup (e.g., Fi1d, x405A, x465A, GCaM) 

In [4]:
import tdt

folder = r"C:\Users\shahd\Box\Awake Project\Maze data\Auditory experiments\dopamine recordings\dopaminergic recordings\MickeyMouse-251205-113917"
data = tdt.read_block(folder)
print(data.streams.keys())
print(data.epocs.keys())

read from t=0s to t=3933.27s
dict_keys(['_405A', '_465A', '_560B', '_405C', '_465C', '_560D', 'Fi1r', 'Fi2r'])
dict_keys(['MTL_', 'Tick'])


In [1]:
import tdt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# --- 1. USER SETTINGS ---
block_path = r"C:\Users\shahd\Box\Awake Project\Maze data\Auditory experiments\dopamine recordings\dopaminergic recordings\MickeyMouse-251205-113917"

# Time frame in seconds
start_time =  (18 * 60) + 15
end_time = (19 * 60) + 20

# Stream and TTL names
stream_name = '_465A' # Your primary green signal
ttl_name = 'Tick'     # CHANGE THIS to your actual TTL key (e.g., 'PtA0', 'Tick')

# Video settings
video_filename = 'photometry_with_TTLs.mp4'
fps = 30 
duration = 5 

# --- 2. LOAD DATA & TTLS ---
print(f"Loading data from {start_time}s to {end_time}s...")
data = tdt.read_block(block_path, t1=start_time, t2=end_time)

# Extract continuous trace
trace = data.streams[stream_name].data
time = np.linspace(start_time, end_time, len(trace))

# Extract TTL onsets and filter for the ones inside our time window
all_ttls = data.epocs[ttl_name].onset
ttls_in_window = [t for t in all_ttls if start_time <= t <= end_time]

# --- 3. SET UP THE PLOT ---
fig, ax = plt.subplots(figsize=(10, 4))
ax.set_xlim(start_time, end_time)
ax.set_ylim(np.min(trace) - 0.05*abs(np.min(trace)), np.max(trace) + 0.05*abs(np.max(trace)))
ax.set_xlabel('Time (s)')
ax.set_ylabel('Fluorescence (a.u.)')
ax.set_title(f'Fiber Photometry Trace with {ttl_name} Events')

# Draw static vertical lines for the TTLs in the background
for t in ttls_in_window:
    ax.axvline(x=t, color='red', linestyle='--', alpha=0.7, label='TTL Event' if t == ttls_in_window[0] else "")

# Add a legend if we have TTLs
if ttls_in_window:
    ax.legend(loc='upper right')

# Create the empty line object for the animation
line, = ax.plot([], [], lw=2, color='green')

# --- 4. ANIMATE AND SAVE ---
total_frames = fps * duration
data_points_per_frame = len(trace) // total_frames

def init():
    line.set_data([], [])
    return line,

def animate(i):
    idx = min((i + 1) * data_points_per_frame, len(trace))
    line.set_data(time[:idx], trace[:idx])
    return line,

print(f"Found {len(ttls_in_window)} TTL events in this window. Generating animation...")
ani = animation.FuncAnimation(fig, animate, init_func=init, 
                              frames=total_frames, interval=1000/fps, blit=True)

writer = animation.FFMpegWriter(fps=fps, metadata=dict(artist='Me'), bitrate=1800)
ani.save(video_filename, writer=writer)

print(f"Done! Video saved as {video_filename}")
plt.close()

Loading data from 1095s to 1160s...
read from t=1095s to t=1160s
Found 65 TTL events in this window. Generating animation...
Done! Video saved as photometry_with_TTLs.mp4


In [2]:
import tdt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# --- 1. USER SETTINGS ---
block_path = r"C:\Users\shahd\Box\Awake Project\Maze data\Auditory experiments\dopamine recordings\dopaminergic recordings\MickeyMouse-251205-113917"

# Time frame of the overall video in seconds
start_time =  (18 * 60) + 15
end_time = (19 * 60) + 20   

# Sliding window settings
window_length = 5.0  # How many seconds of data to show on screen at once

# Stream and TTL names
stream_name = '_465A' # Your primary green signal
ttl_name = 'Tick'     # CHANGE THIS to your actual TTL key (e.g., 'PtA0', 'Tick')

# Video settings
video_filename = 'photometry_sliding_window.mp4'
fps = 30 # Pro-tip: Match this to your behavior camera's FPS!

# --- 2. LOAD DATA & TTLS ---
print(f"Loading data from {start_time}s to {end_time}s...")
data = tdt.read_block(block_path, t1=start_time, t2=end_time)

# Extract trace
trace = data.streams[stream_name].data
time = np.linspace(start_time, end_time, len(trace))

# Extract TTL onsets AND offsets
onsets = data.epocs[ttl_name].onset
offsets = data.epocs[ttl_name].offset

# Safeguard: if the recording stopped mid-TTL, TDT might drop the last offset
if len(offsets) < len(onsets):
    offsets = np.append(offsets, time[-1])

# --- 3. SET UP THE PLOT (DARK MODE) ---
plt.style.use('dark_background') # Makes everything black/white appropriately
fig, ax = plt.subplots(figsize=(10, 4))

ax.set_ylim(np.min(trace) - 0.05*abs(np.min(trace)), np.max(trace) + 0.05*abs(np.max(trace)))
ax.set_xlabel('Time (s)')
ax.set_ylabel('Fluorescence (a.u.)')
ax.set_title(f'Auditory Stimulus Responses')

# Remove top and right borders for a cleaner look
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Draw shaded regions for the TTLs
for on, off in zip(onsets, offsets):
    # Only draw if the TTL falls within our video timeframe
    if on < end_time and off > start_time:
        # Shaded gray box for the duration of the sound
        ax.axvspan(on, off, color='white', alpha=0.2, lw=0) 

# Create the line object (Green trace)
line, = ax.plot([], [], lw=2, color='#00FF00') # Bright neon green

# --- 4. ANIMATE AND SAVE ---
duration = end_time - start_time
total_frames = int(fps * duration)
data_points_per_frame = len(trace) // total_frames

def init():
    line.set_data([], [])
    return line,

def animate(i):
    # Calculate the current "leading edge" of time
    current_time = start_time + (i / fps)
    
    # Calculate the trailing edge of the window
    window_start = max(start_time, current_time - window_length)
    
    # Update the x-axis limits to create the "sliding" effect
    ax.set_xlim(window_start, current_time)
    
    # Figure out how much data to plot up to this frame
    idx = min((i + 1) * data_points_per_frame, len(trace))
    
    # Optional: to save memory, only plot data within the current window
    # But for short videos (a few mins), plotting time[:idx] is totally fine
    line.set_data(time[:idx], trace[:idx])
    
    return line,

print("Generating sliding window animation... this will take a moment.")
# blit=False is REQUIRED here because we are dynamically changing the axes limits
ani = animation.FuncAnimation(fig, animate, init_func=init, 
                              frames=total_frames, interval=1000/fps, blit=False)

writer = animation.FFMpegWriter(fps=fps, bitrate=2500)
ani.save(video_filename, writer=writer)

print(f"Done! Video saved as {video_filename}")
plt.close()

Loading data from 1095s to 1160s...
read from t=1095s to t=1160s
Generating sliding window animation... this will take a moment.


C:\Users\shahd\AppData\Local\Temp\ipykernel_26072\2873379336.py:80: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  ax.set_xlim(window_start, current_time)


Done! Video saved as photometry_sliding_window.mp4


In [40]:
import pandas as pd

file = r"C:\Users\shahd\Box\Awake Project\Maze data\Auditory experiments\dopamine recordings\vocalisations\time_2025-12-05_11_41_09mouseK30_5\mouseK30_5_vocalisations_detailed_visits.csv"

df = pd.read_csv(file)
# df['sound_on_time'] = pd.to_datetime(df['sound_on_time'],unit='s')
# df['sound_off_time'] = pd.to_datetime(df['sound_off_time'],unit='s')

t0 = df.iat[9,4]
t0= t0 +3



df['sound_on_time'] = df['sound_on_time'] - t0
df['sound_off_time'] = df['sound_off_time'] - t0

on_minutes = (df['sound_on_time'] // 60).astype(int).astype(str)
on_seconds = (df['sound_on_time'] % 60).astype(int).astype(str)

# 2. Pad the seconds with a leading zero (so 5 becomes "05")
on_seconds = on_seconds.str.zfill(2)

# 3. Combine them with a colon
df["on_minutes"] = on_minutes + ":" + on_seconds

off_minutes = (df['sound_off_time'] // 60).astype(int).astype(str)
off_seconds = (df['sound_off_time'] % 60).astype(int).astype(str)

# 2. Pad the seconds with a leading zero (so 5 becomes "05")
off_seconds = off_seconds.str.zfill(2)

# 3. Combine them with a colon
df["off_minutes"] = off_minutes + ":" + off_seconds

pd.set_option('display.max_rows', None)
df #.sort_values(by=['time_spent_seconds'], ascending=False)


,trial_ID,ROI_visited,stimulus,sound_on_time,sound_off_time,time_spent_seconds,on_minutes,off_minutes
0,1,entrance2,Unknown_Stimulus,-3.350220,-3.090255,0.259966,-1:56,-1:56
1,1,ROI3,frequency:0 | interval_type:silent_trial | int...,-3.345197,-3.090255,0.254943,-1:56,-1:56
2,1,ROI4,frequency:0 | interval_type:silent_trial | int...,-3.342810,-3.090255,0.252555,-1:56,-1:56
3,1,ROI5,frequency:0 | interval_type:silent_trial | int...,-3.340811,-3.090255,0.250556,-1:56,-1:56
4,1,ROI6,frequency:0 | interval_type:silent_trial | int...,-3.336809,-3.076107,0.260703,-1:56,-1:56
5,1,entrance1,Unknown_Stimulus,-3.352890,-3.005999,0.346891,-1:56,-1:56
6,1,ROI1,frequency:0 | interval_type:silent_trial | int...,-3.348221,-3.004993,0.343227,-1:56,-1:56
7,1,ROI2,frequency:0 | interval_type:silent_trial | int...,-3.346220,-3.003002,0.343219,-1:56,-1:56
8,1,ROI7,frequency:0 | interval_type:silent_trial | int...,-3.332178,-3.001003,0.331175,-1:56,-1:56
9,1,ROI8,frequency:0 | interval_type:silent_trial | int...,-3.329792,-3.000000,0.329792,-1:56,-1:57


In [35]:
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]



In [41]:
import imageio_ffmpeg
import subprocess as sp

In [3]:
%pip install moviepy

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

   -------------------------- ------------- 2/3 [moviepy]
   ---------------------------------------- 3/3 [moviepy]

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import subprocess as sp
import imageio_ffmpeg
ffmpeg_path = imageio_ffmpeg.get_ffmpeg_exe()

start_time =  (18 * 60) + 15
end_time = (19 * 60) + 20
video = r"C:\Users\shahd\Box\Awake Project\Maze data\Auditory experiments\dopamine recordings\vocalisations\time_2025-12-05_11_41_09mouseK30_5\mouseK30_5_time_2025-12-05_11_41_09.mp4"

target_path = r"C:\Users\shahd\Box\Awake Project\Maze data\Auditory experiments\dopamine recordings\vocalisations\time_2025-12-05_11_41_09mouseK30_5\c.mp4"

# ffmpeg_extract_subclip(video, start_time, end_time, target_path)
sp.call([ffmpeg_path, '-loglevel', 'quiet', '-ss', str(start_time), '-to', str(end_time), '-i', video, '-c', 'copy', f"{target_path}"])

0

In [8]:
input_video = r"C:\Users\shahd\Box\Awake Project\Maze data\Auditory experiments\dopamine recordings\vocalisations\time_2025-12-05_11_41_09mouseK30_5\c.mp4"
output_video = r"C:\Users\shahd\Box\Awake Project\Maze data\Auditory experiments\dopamine recordings\vocalisations\time_2025-12-05_11_41_09mouseK30_5\c_withrois.mp4"

In [10]:
import cv2
import random
import numpy as np

# --- CUSTOM ROI DRAWING TOOL (Unchanged) ---
def draw_custom_rois(frame, max_rois=8):
    rois = []
    drawing = False
    start_pt = (-1, -1)
    temp_frame = frame.copy()
    display_frame = frame.copy()

    def mouse_callback(event, x, y, flags, param):
        nonlocal drawing, start_pt, temp_frame, display_frame, rois
        if len(rois) >= max_rois: return 

        if event == cv2.EVENT_LBUTTONDOWN:
            drawing = True
            start_pt = (x, y)
        elif event == cv2.EVENT_MOUSEMOVE:
            if drawing:
                temp_frame = display_frame.copy()
                cv2.rectangle(temp_frame, start_pt, (x, y), (255, 0, 0), 2)
                cv2.imshow("Draw 8 ROIs", temp_frame)
        elif event == cv2.EVENT_LBUTTONUP:
            drawing = False
            end_pt = (x, y)
            x_min, y_min = min(start_pt[0], end_pt[0]), min(start_pt[1], end_pt[1])
            w, h = abs(start_pt[0] - end_pt[0]), abs(start_pt[1] - end_pt[1])

            if w > 5 and h > 5:
                rois.append((x_min, y_min, w, h))
                cv2.rectangle(display_frame, (x_min, y_min), (x_min + w, y_min + h), (0, 255, 0), 2)
                cv2.putText(display_frame, f"ROI {len(rois)}", (x_min, y_min - 5), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                cv2.imshow("Draw 8 ROIs", display_frame)

            if len(rois) == max_rois:
                print("\n8 ROIs drawn! Press any key to start processing...")

    cv2.imshow("Draw 8 ROIs", display_frame)
    cv2.setMouseCallback("Draw 8 ROIs", mouse_callback)

    while True:
        key = cv2.waitKey(1) & 0xFF
        if len(rois) >= max_rois:
            cv2.waitKey(0) 
            break
        if key == 27: 
            break

    cv2.destroyWindow("Draw 8 ROIs")
    return rois

# --- GENERATE EMPTY BACKGROUND (Unchanged) ---
def get_empty_background(cap, total_frames, num_samples=30):
    print(f"Calculating empty background from {num_samples} random frames...")
    frames = []
    for _ in range(num_samples):
        fid = random.randint(0, total_frames - 1)
        cap.set(cv2.CAP_PROP_POS_FRAMES, fid)
        ret, frame = cap.read()
        if ret:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frames.append(gray)
            
    median_frame = np.median(frames, axis=0).astype(dtype=np.uint8)
    return median_frame

# --- MAIN PROCESSING WITH YOUR PATCH HACK ---
def process_video_rois(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, random.randint(0, total_frames - 1))
    ret, random_frame = cap.read()
    
    # 1. Draw ROIs (DRAW THE PROBLEMATIC ONE FIRST!)
    print("\nIMPORTANT: Draw the ROI where the mouse spends the most time FIRST!")
    rois = draw_custom_rois(random_frame, max_rois=8)
    
    if len(rois) == 0: return

    # 2. Get the median background
    empty_bg = get_empty_background(cap, total_frames)

    # 3. THE FIX: Patch ROI 1 using the other 7 ROIs
    if len(rois) == 8:
        other_rois_pixels = []
        # Gather all pixels from ROI 2 through 8
        for (x, y, w, h) in rois[1:]:
            patch = empty_bg[y:y+h, x:x+w]
            other_rois_pixels.extend(patch.flatten())
            
        # Calculate the average color
        avg_bg_color = int(np.mean(other_rois_pixels))
        
        # Paint over ROI 1 (rois[0]) with this average color
        x1, y1, w1, h1 = rois[0]
        empty_bg[y1:y1+h1, x1:x1+w1] = avg_bg_color
        print(f"Patched ROI 1 background with average color: {avg_bg_color}")
        
        # Optional: Show the patched background so you can verify it worked
        cv2.imshow("Patched Background - Press any key to start video", empty_bg)
        cv2.waitKey(0)
        cv2.destroyWindow("Patched Background - Press any key to start video")

    # 4. Setup Video Writer
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    print("Processing video...")

    # 5. Process video
    while True:
        ret, frame = cap.read()
        if not ret: break 

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        diff = cv2.absdiff(empty_bg, gray_frame)
        _, binary_mask = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

        for (x, y, w, h) in rois:
            roi_mask = binary_mask[y:y+h, x:x+w]
            white_pixels = cv2.countNonZero(roi_mask)
            total_pixels = w * h
            
            occupancy_ratio = white_pixels / total_pixels
            
            if occupancy_ratio > 0.50:
                color = (0, 255, 0) # Green
            else:
                color = (0, 0, 255) # Red
                
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Done! Video saved to {output_path}")

# --- RUN THE CODE ---


process_video_rois(input_video, output_video)


IMPORTANT: Draw the ROI where the mouse spends the most time FIRST!

8 ROIs drawn! Press any key to start processing...
Calculating empty background from 30 random frames...
Patched ROI 1 background with average color: 232
Processing video...
Done! Video saved to C:\Users\shahd\Box\Awake Project\Maze data\Auditory experiments\dopamine recordings\vocalisations\time_2025-12-05_11_41_09mouseK30_5\c_withrois.mp4
